# モデルの学習・テストデータによる出力

In [106]:
# 設定
import sys
sys.path.append('../src/')      # import 用のパスの追加
sys.dont_write_bytecode = True  # __pycache__ 生成を防ぐ

# 標準
import pandas as pd
import numpy as np

# データ加工用
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

# モデル学習
from sklearn.model_selection import train_test_split # データセット分割用

# メトリクス
from sklearn.metrics import mean_squared_error # モデル評価用(平均二乗誤差)
from sklearn.metrics import r2_score # モデル評価用(決定係数)

# モデル
import lightgbm as lgb #LightGBM
from lightgbm_tips import ModelExtractionCallback


### データの読み込み & 目的変数の作成

In [107]:
train = pd.read_csv('../data/processed_train_df.csv', encoding = 'SHIFT-JIS')

input_Y = train['mode_price'].to_numpy()
train_X = train.drop('mode_price', axis = 1)

### 説明変数の作成

* todo
  * 定性的な情報をカテゴリ変数に変換

In [108]:
# 日付をindexに設定
if 'date' in train_X.columns:
    train_X.set_index('date', inplace=True)

# 連続変数を標準化
scaler = StandardScaler()
Continuous_train_array = scaler.fit_transform(train_X.select_dtypes(['float', 'int']))

# カテゴリ変数の作成
Label_dict = defaultdict(LabelEncoder)
Discrete_train_array = train_X.select_dtypes(exclude=['float', 'int']).apply(lambda x: Label_dict[x.name].fit_transform(x), axis =0).to_numpy()

# 入力変数の作成
input_X = np.concatenate([Continuous_train_array, Discrete_train_array], axis = 1)

# 変換内容の確認
print(input_X.shape == train_X.shape)

True


### トレーニングデータ,テストデータの分割

In [109]:
X_train, X_test, y_train, y_test = train_test_split(input_X, input_Y, test_size=0.20, random_state=2)

### モデルの学習


#### 一般的なモデルの学習方法

In [110]:
# モデルの学習
model = lgb.LGBMRegressor() # モデルのインスタンスの作成
model.fit(X_train, y_train) # モデルの学習

# テストデータの予測
y_pred = model.predict(X_test)

#### CVを使った学習

In [111]:
# LightGBM 用のデータセット表現に直す
lgb_train = lgb.Dataset(X_train, y_train)

# 学習済みモデルを取り出すためのコールバックを用意する
extraction_cb = ModelExtractionCallback()
callbacks = [
    extraction_cb,
]

# データセットを 5-Fold CV で学習する
lgbm_params = {
    'objective': 'regression',
    'verbose': -1
    }

# 交差検証の結果を確認する
lgb.cv(lgbm_params,
        lgb_train,
        num_boost_round=1000,
        early_stopping_rounds=10,
        nfold=3,
        shuffle=True,
        stratified=False,
        seed=42,
        callbacks=callbacks,
        )

# コールバックのオブジェクトから学習済みモデルを取り出す
proxy = extraction_cb.boosters_proxy
boosters = extraction_cb.raw_boosters
best_iteration = extraction_cb.best_iteration

In [112]:
# 各モデルの推論結果を Averaging する場合
y_pred_proba_list = proxy.predict(X_test, num_iteration=best_iteration)
y_pred = np.array(y_pred_proba_list).mean(axis=0)

# 精度計算
accuracy = np.sqrt(np.mean(((y_pred - y_test) / y_test)**2))*100
print('Averaging accuracy:', accuracy)

Averaging accuracy: 55.99948823657205


In [113]:
# 各モデルで個別に推論する場合
for i, booster in enumerate(boosters):
    y_pred = booster.predict(X_test,num_iteration=best_iteration)
    
    accuracy = accuracy = np.sqrt(np.mean(((y_pred - y_test) / y_test)**2))*100
    print('Model {0} accuracy: {1}'.format(i, accuracy))

Model 0 accuracy: 63.301106285999765
Model 1 accuracy: 64.54793254737056
Model 2 accuracy: 59.615788267452594
